In [1]:
data_path = '/Users/artemvolkov/Desktop/Machine_learning_and_predictive_analysis/Lecture_1'

In [28]:
import pandas as pd
import numpy as np
import os
import statsmodels.api as sm

/Users/artemvolkov/opt/anaconda3/envs/myenv/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [92]:
data = pd.read_csv(os.path.join(data_path,"test_sample.csv"))
X = data.iloc[:,1:]
Y = data['Y']
Xs = ["X%i"% (j-1) for j in range(2,493)]

Fitting linear models with predictors' numbers from 2 to 491

In [110]:
lms_unordered = [sm.OLS(Y, sm.add_constant(X.iloc[:,:j])).fit() for j in range(2,492)]

Let us find determination coefficient for each model and find one with the smallest number of predictors, where it reaches level greater than 90%.

In [123]:
r2s = np.array([model.rsquared for model in lms_unordered])
N_orig = np.argmax(r2s > 0.9)+2
N_orig

334

## The least features number enough for $R^2 >0.9$ is 334.

Now let us apply PCA method for our features set, then fit models using the resulting feature set and find the least number of primary components to exceed the 0.9 level for $R^2$.

In [124]:
from sklearn.decomposition import PCA

In [125]:
xPCA = PCA(n_components = data.shape[1]-1)
xPCA.fit(X)

PCA(n_components=491)

In [126]:
lms_pca = [sm.OLS(Y, sm.add_constant(np.dot(X, xPCA.components_.T)[:,:j])).fit() for j in range(2, 492)]

In [128]:
r2s = np.array([model.rsquared for model in lms_pca])
N_PCA = np.argmax(r2s>0.9)+2
N_PCA

245

## We see, that the least numer of PCs is 245, which is much less than basic number of components reached in the previous step.

Let us sort PCAs and then reestimate LMs on ordered PCA set.

Defining function of calculation of relative importance measures.

In [134]:
def rel_imp_me(X, y): 
    names = X.columns
    ser = pd.Series(index = names, dtype = np.float64)
    lm0 = sm.OLS(y, sm.add_constant(X)).fit()
    for c in names:
        lm = sm.OLS(y, sm.add_constant(X[names.drop(c)])).fit()
        ser[c] = lm0.rsquared - lm.rsquared 
    res = pd.DataFrame(columns =['last', 'first', 'betasq', 'pratt'], index=names)
    res['last'] = ser
    corr = X.apply(lambda x: np.corrcoef(y,x)[0,1], axis=0)
    res['first'] = corr**2
    sx = X.std()
    res['betasq'] = (lm0.params[names] * sx / np.std(y))**2
    res['pratt'] = (lm0.params[names] * sx / np.std(y)) * corr
    return res

In [135]:
factorScores = pd.DataFrame(np.dot(X, xPCA.components_.T), columns = ["PC%i"%(j+1) for j in range(491)])
metrics_PCA = rel_imp_me(factorScores, Y)

In [136]:
ranked_pca = metrics_PCA["first"].rank(ascending=False, method='first')

In [137]:
metrics_PCA_sort = pd.DataFrame({"Factors" : ranked_pca.index,
                                 "Rank" : ranked_pca.values}).sort_values(by="Rank")
ordered_factor_scores = pd.DataFrame(factorScores, columns=metrics_PCA_sort["Factors"])

In [138]:
lms_pca = [sm.OLS(Y, sm.add_constant(ordered_factor_scores.iloc[:,:j])).fit() for j in range(2, 492)]

In [139]:
r2s = np.array([model.rsquared for model in lms_pca])
N_PCA = np.argmax(r2s>0.9)+2
N_PCA

138

## After sorting PCs we can see, that the least number has dropped even more, and now it is 138.
## Thus, we have reduced the dimension of feature space for 0.9 determination level from 334 to 138 components.

In [145]:
lms_pca[N_PCA-2].rsquared

0.9009692232515636